# Project: Wrangling and Analyze Data

Importing Libraries

In [1]:
import requests #download data
import numpy as np #array functions
import pandas as pd #data handling
import tweepy #twitter api
import json #handle json data
import matplotlib.pyplot as plt #data visualization
import seaborn as sns #data visualization
import re #text processing
import time

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly downloading the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv) and displaying the top 5 rows

**1. Loading the Dataset**

In [4]:
twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")
twitter_archive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


**2. Image Predictions**

First defining the url file and then using the request.get function, and write the content of the response to a new file. 

In [5]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

#get response
response = requests.get(url)

#write return to an image
with open("image_predictions.tsv", mode = "wb") as file:
    file.write(response.content)

In [6]:
twitter_predict = pd.read_csv("image_predictions.tsv", sep='\t')
twitter_predict.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


**3. Gathering additional data with the Twitter API**

In [7]:
access_token = "1556583358372642816-EBDvb4QsFo4xeYFdnT8zO0neKhTuq9"
access_secret = "lNUpIXE9ENNUjCbVbFDIqQ5GS9DavCKgeYOBtuLL5928W"
consumer_key = "aPBkWyvtzGV41yl17OLMlXeDa"
consumer_secret = "vk9mGd5ls7CUH2Mq7o5auhLoxuCwODmmwxcHm040bZBPjpBWo3"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAAfzfgEAAAAA9CDd58pGvknfdOlSG3Dn%2FGAawtI%3DEUEcRid0Z48ojEMLKBzlg6AfXsRC7sBHNYqjLSWtZfR3xiOS54"

In [12]:
print(type(bearer_token))

<class 'str'>


In [23]:
print(type(consumer_key))

<class 'str'>


In [8]:
client_id="SklVakRsZkxKNi1QOFAwdUQ1WUo6MTpjaQ"
client_secret="p3GGBNbylckZbFqtv5yazdCv4cgh-tmORMedgujFZ8sSx3mFBr"

Getting all IDs in Twitter Dataset

In [22]:
twitter_ids = list(twitter_archive.tweet_id.unique())

# Tweet IDs for which to gather additional data via Twitter's API
twitter_ids = twitter_archive.tweet_id.values
len(twitter_ids)

2356

In [42]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [47]:
auth = tweepy.OAuthHandler(client_id, client_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

additional source:  https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id

In [48]:
 # Creating a for loop to add the tweets to a new line of the datafile 
with open('tweet_json.txt', 'a', encoding='utf8') as f:
    for id in twitter_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, f)
            f.write('\n')
        except:
            continue

In [53]:
twitter_list = []

# Read the .txt file line by line into a list of dictionaries
for line in open('tweet_json.txt', 'r'):
    twitter_data = json.loads(line)
    twitter_list.append({'tweet_id': twitter_data['id_str'],
                        'retweet_count': twitter_data['retweet_count'],
                        'favorite_count': twitter_data['favorite_count'],
                        'followers_count': twitter_data['user']['followers_count']})
    

In [50]:
 # Convert the list of dictionaries to a pandas DataFrame
twitter_df = pd.DataFrame(twitter_list, columns = ['tweet_id', 'retweet_count', 'favorite_count', 'followers_count'])

In [52]:
twitter_df.head(5)

,tweet_id,retweet_count,favorite_count,followers_count


## attempt three

In [14]:
!pip install --upgrade tweepy==4.6.0
import tweepy

    100% |████████████████████████████████| 71kB 3.8MB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 4.3MB/s ta 0:00:011
    100% |████████████████████████████████| 153kB 15.0MB/s ta 0:00:01
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4
  Found existing installation: oauthlib 2.0.6
    Uninstalling oauthlib-2.0.6:
      Successfully uninstalled oauthlib-2.0.6
  Found existing installation: requests-oauthlib 0.8.0
    Uninstalling requests-oauthlib-0.8.0:
      Successfully uninstalled requests-oauthlib-0.8.0
  Found existing installation: tweepy 3.5.0
    Uninstalling tweepy-3.5.0:
      Successfully uninstalled tweepy-3.5.0


In [17]:
import tweepy 

# defining bearer token received through developed access for twitter to get the twitter data via Client
bearer_token='AAAAAAAAAAAAAAAAAAAAAAfzfgEAAAAAASzZc%2FA2shpjxfzSMAcEk0aMfbY%3DAntBgueRHyGMxO6ag3FrHbY5Uqx48BCBG84ZQQSmVnYHqPE7sH'

# Initialising Client
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

tweet_ids=twitter_archive.tweet_id.values

#trying out code for one example tweet ID
example_tweet=tweet_ids[0]

print(f'Quering API for tweets: {example_tweet}\n') 

# retweet and favourites are in public_metrics
tweet_fields=['created_at', 'id', 'public_metrics','source', 'text']

# get information on individual tweet - example_tweet
temp=client.get_tweet(id=example_tweet, tweet_fields=tweet_fields)

# store information in .data
print(f'{temp.data}\n') 

#if temp.data.get('public_metrics'):
 #   print(f"{temp.data['public_metrics']=}") 


AttributeError: module 'tweepy' has no attribute 'Client'

In [19]:
# running the code for whole twitter ids
# Initialising Client
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

tweet_ids=twitter_archive.tweet_id.values

print(f'Quering API for tweets: {example_tweet}\n') 

# retweet and favourites are in public_metrics
tweet_fields=['created_at', 'id', 'public_metrics','source', 'text']

# get information on individual tweet - example_tweet
temp=client.get_tweet(id=tweet_ids, tweet_fields=tweet_fields)

# store information in .data
print(f'{temp.data}\n') 

#if temp.data.get('public_metrics'):
 #   print(f"{temp.data['public_metrics']=}") 


AttributeError: module 'tweepy' has no attribute 'Client'

In [56]:
# store data
tweet_data={}
if temp.data.get('public_metrics'):
    for key, value in temp.data['public_metrics'].items():
        #store the results
        tweet_data[key]=value
        
tweet_data

{'retweet_count': 6982,
 'reply_count': 139,
 'like_count': 33742,
 'quote_count': 219}

In [30]:
!pip install --upgrade tweepy==4.6.0
import tweepy

Requirement already up-to-date: tweepy==4.6.0 in /opt/conda/lib/python3.6/site-packages (4.6.0)


In [20]:
import tweepy
import time
import json

bearer_token=['AAAAAAAAAAAAAAAAAAAAAAfzfgEAAAAA9CDd58pGvknfdOlSG3Dn%2FGAawtI%3DEUEcRid0Z48ojEMLKBzlg6AfXsRC7sBHNYqjLSWtZfR3xiOS54']
# initialise .Client()
client=tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

info=['public_metrics']

tweet_info=[]

# Loop through each tweet_id
for ind, tweet_id in enumerate(twitter_archive.tweet_id):
    public_metrics={}
    # store tweet_id
    public_metrics['tweet_id']=tweet_id
    # Basic request per tweet_id
    temp_data = client.get_tweet(id=tweet_id, tweet_fields=info)
    if temp_data.data:
        if temp_data.data.get('public_metrics'):
            # Loop through the public metrics dictionary to get the keys/values that are embedded
            for key, value in temp_data.data['public_metrics'].items():
                public_metrics[key]=value
    tweet_info.append(public_metrics)

AttributeError: module 'tweepy' has no attribute 'Client'

In [34]:
client.get_me()

TypeError: Consumer key must be string or bytes, not NoneType

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [94]:
twitter_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1315,706681918348251136,NaN,NaN,2016-03-07 03:24:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Katie. She's a Mitsubishi Hufflep...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/706681918...,12,10,Katie,None,None,None,None
1355,703611486317502464,NaN,NaN,2016-02-27 16:03:45 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Scooter. He's experiencing the pupper equ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/703611486...,10,10,Scooter,None,None,pupper,None
1215,715009755312439296,NaN,NaN,2016-03-30 02:56:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Siba. She's remarkably mobile. Very sl...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715009755...,12,10,Siba,None,None,None,None
453,818588835076603904,NaN,NaN,2017-01-09 22:42:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Chelsea. She forgot how...,7.735476e+17,4.196984e+09,2016-09-07 15:44:53 +0000,https://twitter.com/dog_rates/status/773547596...,11,10,Chelsea,None,None,pupper,None
362,829449946868879360,NaN,NaN,2017-02-08 22:00:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a stressed doggo. Had a long day. Many ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/829449946...,11,10,None,doggo,None,None,None


In [103]:
twitter_predict.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1018,710117014656950272,https://pbs.twimg.com/media/CdrXp9dWoAAcRfn.jpg,2,toy_poodle,0.802092,True,miniature_poodle,0.111647,True,cocker_spaniel,0.062866,True
806,691820333922455552,https://pbs.twimg.com/media/CZnW7JGW0AA83mn.jpg,1,minivan,0.332756,False,sports_car,0.129452,False,limousine,0.073936,False
1688,815390420867969024,https://pbs.twimg.com/media/C1DZQiTXgAUqgRI.jpg,1,restaurant,0.279846,False,toyshop,0.091429,False,paper_towel,0.046147,False
43,666776908487630848,https://pbs.twimg.com/media/CUDeDoWUYAAD-EM.jpg,1,seat_belt,0.375057,False,miniature_pinscher,0.167175,True,Chihuahua,0.086951,True
1643,808001312164028416,https://pbs.twimg.com/media/CzaY5UdUoAAC91S.jpg,1,Labrador_retriever,0.730959,True,Staffordshire_bullterrier,0.130726,True,American_Staffordshire_terrier,0.028853,True


In [ ]:
twitter_tweets.sample(5)

In [99]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [104]:
twitter_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [ ]:
twitter_tweets.info()

In [105]:
list(twitter_predict)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

In [106]:
list(twitter_archive)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [ ]:
list(twitter_tweets)

In [ ]:
all_columns = pd.Series(list(twitter_archive) + list(twitter_predict) + list(twitter_tweets))
all_columns[all_columns.duplicated()]

In [107]:
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [109]:
twitter_predict.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [ ]:
twitter_tweets.describe()

In [110]:
twitter_archive.name.value_counts()

None          745
a              55
Charlie        12
Oliver         11
Cooper         11
Lucy           11
Lola           10
Tucker         10
Penny          10
Winston         9
Bo              9
Sadie           8
the             8
Daisy           7
Buddy           7
Toby            7
an              7
Bailey          7
Jax             6
Oscar           6
Scout           6
Jack            6
Dave            6
Milo            6
Rusty           6
Bella           6
Koda            6
Leo             6
Stanley         6
Oakley          5
             ... 
Timber          1
Ralphy          1
Yoda            1
Durg            1
Alf             1
Grady           1
Harnold         1
Rumpole         1
Karma           1
Kramer          1
Longfellow      1
Jarvis          1
Ozzie           1
Vince           1
Jerome          1
Godi            1
Lacy            1
Spark           1
Damon           1
Spencer         1
Petrick         1
Skittle         1
Fiji            1
Gordon          1
Lucia     

In [128]:
twitter_archive['is_duplicated'].sum()

0

In [136]:
twitter_archive[tweet_id.duplicated()]

AttributeError: 'int' object has no attribute 'duplicated'

In [112]:
twitter_predict.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
malamute                      30
cocker_spaniel                30
French_bulldog                26
miniature_pinscher            23
Chesapeake_Bay_retriever      23
seat_belt                     22
German_shepherd               20
Siberian_husky                20
Staffordshire_bullterrier     20
Cardigan                      19
web_site                      19
Maltese_dog                   18
Shetland_sheepdog             18
teddy                         18
beagle                        18
Eskimo_dog                    18
Lakeland_terrier              17
Rottweiler                    17
Shih-Tzu                      17
Italian_greyhound             16
kuvasz                        16
          

In [137]:
twitter_predict[twitter_predict.tweet_id.duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [113]:
list(twitter_archive.text)[:5]

["This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 "This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",
 'This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB',
 'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ',
 'This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f']

In [114]:
twitter_archive.timestamp

0       2017-08-01 16:23:56 +0000
1       2017-08-01 00:17:27 +0000
2       2017-07-31 00:18:03 +0000
3       2017-07-30 15:58:51 +0000
4       2017-07-29 16:00:24 +0000
5       2017-07-29 00:08:17 +0000
6       2017-07-28 16:27:12 +0000
7       2017-07-28 00:22:40 +0000
8       2017-07-27 16:25:51 +0000
9       2017-07-26 15:59:51 +0000
10      2017-07-26 00:31:25 +0000
11      2017-07-25 16:11:53 +0000
12      2017-07-25 01:55:32 +0000
13      2017-07-25 00:10:02 +0000
14      2017-07-24 17:02:04 +0000
15      2017-07-24 00:19:32 +0000
16      2017-07-23 00:22:39 +0000
17      2017-07-22 16:56:37 +0000
18      2017-07-22 00:23:06 +0000
19      2017-07-21 01:02:36 +0000
20      2017-07-20 16:49:33 +0000
21      2017-07-19 16:06:48 +0000
22      2017-07-19 03:39:09 +0000
23      2017-07-19 00:47:34 +0000
24      2017-07-18 16:08:03 +0000
25      2017-07-18 00:07:08 +0000
26      2017-07-17 16:17:36 +0000
27      2017-07-16 23:58:41 +0000
28      2017-07-16 20:14:00 +0000
29      2017-0

In [129]:
twitter_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [130]:
twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

### Quality issues
1. twitter_archive has some duplicate tweets

2. twitter_archive text column contains hyperlink which we do not need 

3. twitter_archive timestamp is data type object and tweet_id is integer

4. twitter_predict shows several predictions but only one would be needed

5. twitter_archive has unnecessary columns like in_reply_to_status_idm in_reply_to_user_id, retweeted_status_idm source, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp

6. twitter_archive - some rows in column name contain 'None' - missing data

7. twitter_archive - some rows in column name contains non-names values with lower case - like a or quite

8. twitter_tweets columns, despite being numbers, are ‘object’ Dtype and the shared observation is labeled tweet ID 

9. twitter_archive rating_numerator has values ranging from 0 to 1779 and denominator 0-170

### Tidiness issues
1. twitter_tweets and twitter_predict have certain some observations as twitter_archive - they do not need to be separate observations

2. twitter_archive has column headers doggo, flooder, pepper, puppy - these could be better as one variable in one column not as separate columns

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [25]:
# Make copies of original pieces of data
twitter_archive_clean = twitter_archive.copy()
twitter_predict_clean = twitter_predict.copy()
twitter_tweets_clean = twitter_tweets.copy()

NameError: name 'twitter_tweets' is not defined

### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization